# Quantum ESPRESSO Workshop - Part 8: Summary and Exercises

## Workshop Overview

This notebook provides:
1. Complete workflow summary
2. Quick reference for all calculations
3. Practice exercises with solutions
4. Troubleshooting guide
5. Further resources

---

## 1. Complete DFT Workflow Summary

### The Rigorous Approach

```
┌─────────────────────────────────────────────────────────────────┐
│  BEFORE calculating any property, you MUST:                     │
│                                                                 │
│  1. Converge ecutwfc (wavefunction cutoff)                     │
│  2. Converge k-points (Brillouin zone sampling)                │
│  3. Optimize lattice parameter (equilibrium structure)         │
│                                                                 │
│  Only THEN proceed to property calculations:                    │
│  • Band structure                                              │
│  • Density of states                                           │
│  • Phonons, optical properties, etc.                           │
└─────────────────────────────────────────────────────────────────┘
```

### Workflow Diagram

```
                    START
                      │
                      ▼
            ┌─────────────────┐
            │   Get/Create    │
            │ Pseudopotential │
            └────────┬────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │  Ecutwfc Convergence  │◄──── Notebook 03
         │  (vary ecutwfc,       │
         │   fix k-pts, alat)    │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │  K-point Convergence  │◄──── Notebook 04
         │  (fix ecutwfc,        │
         │   vary k-pts)         │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │ Lattice Optimization  │◄──── Notebook 05
         │  (fix ecutwfc, k-pts, │
         │   vary alat → E(V))   │
         └───────────┬───────────┘
                     │
                     ▼
         ┌───────────────────────┐
         │    CONVERGED BASE     │
         │  (ecutwfc, k-pts,     │
         │   optimized alat)     │
         └───────────┬───────────┘
                     │
        ┌────────────┼────────────┐
        │            │            │
        ▼            ▼            ▼
   ┌─────────┐  ┌─────────┐  ┌─────────┐
   │  Band   │  │   DOS   │  │ Phonons │
   │Structure│  │  PDOS   │  │   etc.  │
   └─────────┘  └─────────┘  └─────────┘
    Notebook 06  Notebook 07
```

---

## 2. Quick Reference: Input File Templates

In [ ]:
# Quick reference templates

SCF_TEMPLATE = '''
&CONTROL
    calculation = 'scf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/
&SYSTEM
    ibrav = {ibrav}
    celldm(1) = {celldm1}
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/
&ELECTRONS
    conv_thr = 1.0e-8
    mixing_beta = 0.7
/
ATOMIC_SPECIES
{atomic_species}
ATOMIC_POSITIONS {{crystal}}
{atomic_positions}
K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
'''

BANDS_TEMPLATE = '''
&CONTROL
    calculation = 'bands'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
/
&SYSTEM
    ibrav = {ibrav}
    celldm(1) = {celldm1}
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    nbnd = {nbnd}
/
&ELECTRONS
    conv_thr = 1.0e-8
/
ATOMIC_SPECIES
{atomic_species}
ATOMIC_POSITIONS {{crystal}}
{atomic_positions}
K_POINTS {{crystal_b}}
{kpath}
'''

NSCF_DOS_TEMPLATE = '''
&CONTROL
    calculation = 'nscf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
/
&SYSTEM
    ibrav = {ibrav}
    celldm(1) = {celldm1}
    nat = {nat}
    ntyp = {ntyp}
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'tetrahedra'
    nbnd = {nbnd}
/
&ELECTRONS
    conv_thr = 1.0e-8
/
ATOMIC_SPECIES
{atomic_species}
ATOMIC_POSITIONS {{crystal}}
{atomic_positions}
K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
'''

print("Templates loaded. Use SCF_TEMPLATE, BANDS_TEMPLATE, NSCF_DOS_TEMPLATE")

---

## 3. Key Parameters Quick Reference

### Essential Parameters

| Parameter | Description | Typical Values |
|-----------|-------------|----------------|
| `ecutwfc` | Wavefunction cutoff | 30-80 Ry (check PP) |
| `ecutrho` | Charge density cutoff | 4-12× ecutwfc |
| `conv_thr` | SCF convergence | 1e-6 to 1e-10 Ry |
| `K_POINTS` | BZ sampling | 4-16 per direction |
| `degauss` | Smearing width | 0.01-0.02 Ry |

### Calculation Types

| `calculation` | Purpose | Requires |
|---------------|---------|----------|
| `'scf'` | Self-consistent | Nothing |
| `'relax'` | Ionic relaxation | SCF |
| `'vc-relax'` | Cell + ionic | SCF |
| `'bands'` | Band structure | SCF |
| `'nscf'` | Non-SCF (for DOS) | SCF |

### Ecutrho Guidelines

| Pseudopotential Type | ecutrho |
|----------------------|---------|
| Norm-conserving (NC) | 4× ecutwfc |
| Ultrasoft (US) | 8-12× ecutwfc |
| PAW | 8× ecutwfc |

---

## 4. Converged Parameters for Silicon (PBE)

From our workshop calculations:

In [ ]:
import json
from pathlib import Path

# Load converged parameters
params_file = Path('/home/claude/qe_workshop/converged_parameters.json')

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
    
    print("CONVERGED PARAMETERS FOR SILICON (PBE)")
    print("=" * 50)
    print(json.dumps(params, indent=2))
else:
    print("Run previous notebooks to generate converged parameters.")
    params = {
        'ecutwfc_recommended': 40.0,
        'ecutrho_factor': 8,
        'kpoints_recommended': 8,
        'celldm1_optimized_bohr': 10.33  # Typical PBE value
    }
    print("\nUsing typical values:")
    print(json.dumps(params, indent=2))

---

## 5. Practice Exercises

### Exercise 1: Germanium Band Structure (Intermediate)

**Task**: Calculate the band structure of Germanium using the same workflow.

**Hints**:
- Ge has the same diamond structure as Si
- Experimental lattice parameter: a = 5.658 Å = 10.69 Bohr
- Download Ge pseudopotential from QE website
- Use similar ecutwfc (may need higher for Ge)

In [ ]:
# Exercise 1 Starter Code

# Germanium parameters
GE_PARAMS = {
    'element': 'Ge',
    'mass': 72.64,
    'a_exp_angstrom': 5.658,
    'a_exp_bohr': 10.69,
    'ibrav': 2,  # FCC
    'nat': 2,
    'ntyp': 1,
    'positions': [
        ('Ge', 0.00, 0.00, 0.00),
        ('Ge', 0.25, 0.25, 0.25)
    ]
}

# TODO: Download Ge pseudopotential
# URL: https://pseudopotentials.quantum-espresso.org/upf_files/Ge.pbe-n-kjpaw_psl.1.0.0.UPF

# TODO: Perform ecutwfc convergence test

# TODO: Perform k-point convergence test

# TODO: Optimize lattice parameter

# TODO: Calculate band structure

print("Exercise 1: Complete the Ge band structure calculation")
print(f"Starting parameters: a = {GE_PARAMS['a_exp_bohr']} Bohr")

### Exercise 2: Effect of K-point Sampling on DOS (Easy)

**Task**: Calculate the DOS of Silicon with different k-grids and compare.

**Instructions**:
1. Use the converged ecutwfc and lattice parameter
2. Calculate DOS with k-grids: 8×8×8, 12×12×12, 16×16×16
3. Plot and compare the results
4. Discuss how k-point density affects DOS smoothness

In [ ]:
# Exercise 2 Starter Code

kgrids_to_test = [8, 12, 16]

# TODO: For each k-grid:
#   1. Run NSCF calculation
#   2. Run dos.x
#   3. Parse and store results

# TODO: Plot all DOS on the same figure

# TODO: Analyze the differences

print("Exercise 2: Compare DOS with different k-grids")
print(f"K-grids to test: {kgrids_to_test}")

### Exercise 3: GaAs Band Structure (Advanced)

**Task**: Calculate the band structure of GaAs (zinc blende structure).

**Challenges**:
- Two different atom types (Ga, As)
- Need two pseudopotentials
- Different atomic positions: Ga at (0,0,0), As at (1/4,1/4,1/4)
- Direct band gap (unlike Si)

In [ ]:
# Exercise 3 Starter Code

GAAS_PARAMS = {
    'elements': ['Ga', 'As'],
    'masses': [69.723, 74.922],
    'a_exp_angstrom': 5.653,
    'a_exp_bohr': 10.68,
    'ibrav': 2,  # FCC (zinc blende is FCC with 2-atom basis)
    'nat': 2,
    'ntyp': 2,
    'positions': [
        ('Ga', 0.00, 0.00, 0.00),
        ('As', 0.25, 0.25, 0.25)
    ],
    'exp_band_gap_eV': 1.42  # Direct gap at Γ
}

# Pseudopotential URLs:
# Ga: https://pseudopotentials.quantum-espresso.org/upf_files/Ga.pbe-dn-kjpaw_psl.1.0.0.UPF
# As: https://pseudopotentials.quantum-espresso.org/upf_files/As.pbe-n-kjpaw_psl.1.0.0.UPF

print("Exercise 3: Calculate GaAs band structure")
print(f"Expected band gap: {GAAS_PARAMS['exp_band_gap_eV']} eV (direct)")

---

## 6. Troubleshooting Guide

### Common Errors and Solutions

| Error | Cause | Solution |
|-------|-------|----------|
| SCF not converging | mixing_beta too high | Reduce to 0.3-0.5 |
| SCF not converging | Bad initial guess | Use better starting density |
| Negative frequencies | Structure not relaxed | Relax structure first |
| Memory error | Too many k-points/cutoff | Reduce or use more nodes |
| Wrong band gap | Expected for DFT | Use GW or hybrid functionals |
| Stress not zero | Lattice not optimized | Run vc-relax or E(V) fit |

### Convergence Issues

```python
# If SCF doesn't converge, try these in order:

# 1. Reduce mixing
mixing_beta = 0.3

# 2. Use different mixing mode
mixing_mode = 'local-TF'

# 3. Increase electron_maxstep
electron_maxstep = 200

# 4. Use different diagonalization
diagonalization = 'cg'

# 5. Start from atomic densities
startingpot = 'atomic'
startingwfc = 'atomic'
```

---

## 7. High-Symmetry K-Paths Reference

### FCC (Si, Ge, GaAs, etc.)

```
Γ → X → W → K → Γ → L

Point  Crystal coords
Γ      (0.000, 0.000, 0.000)
X      (0.500, 0.000, 0.500)
W      (0.500, 0.250, 0.750)
K      (0.375, 0.375, 0.750)
L      (0.500, 0.500, 0.500)
```

### BCC (Fe, W, etc.)

```
Γ → H → N → Γ → P → H

Point  Crystal coords
Γ      (0.000, 0.000, 0.000)
H      (0.500, -0.500, 0.500)
N      (0.000, 0.000, 0.500)
P      (0.250, 0.250, 0.250)
```

### HCP (Ti, Zn, etc.)

```
Γ → M → K → Γ → A → L → H → A
```

---

## 8. Further Resources

### Official Documentation
- [Quantum ESPRESSO Documentation](https://www.quantum-espresso.org/Doc/INPUT_PW.html)
- [QE Tutorials](https://www.quantum-espresso.org/resources/tutorials)
- [Pseudopotential Library](https://www.materialscloud.org/discover/sssp/table/efficiency)

### Textbooks
- Richard Martin, "Electronic Structure: Basic Theory and Practical Methods"
- Feliciano Giustino, "Materials Modelling using Density Functional Theory"
- David Sholl & Janice Steckel, "Density Functional Theory: A Practical Introduction"

### Online Courses
- [ICTP QE School](https://www.youtube.com/results?search_query=ictp+quantum+espresso)
- [Materials Project Workshop](https://workshop.materialsproject.org/)

### Pseudopotential Libraries
- [SSSP Library](https://www.materialscloud.org/discover/sssp/table/efficiency) (recommended)
- [PSlibrary](https://dalcorso.github.io/pslibrary/)
- [GBRV](https://www.physics.rutgers.edu/gbrv/)

---

## 9. Workshop Summary

### What We Learned

| Notebook | Topic | Key Takeaway |
|----------|-------|---------------|
| 01 | Setup | Environment verification, pseudopotentials |
| 02 | SCF Basics | Input file structure, output parsing |
| 03 | Ecutwfc | Systematic convergence testing |
| 04 | K-points | BZ sampling, symmetry reduction |
| 05 | Lattice | E(V) fitting, Birch-Murnaghan EOS |
| 06 | Bands | SCF→bands→post-processing workflow |
| 07 | DOS | Total and projected DOS, orbital analysis |
| 08 | Summary | Complete reference, exercises |

### The Golden Rules

1. **Always converge before calculating**: ecutwfc, k-points, structure
2. **Document everything**: Keep records of convergence tests
3. **Compare with experiment**: Validate your setup
4. **Understand limitations**: DFT band gaps, van der Waals, etc.
5. **Use good pseudopotentials**: SSSP library recommended

### Next Steps

After mastering the basics:
1. **Phonon calculations** (ph.x, matdyn.x)
2. **Structural relaxation** (relax, vc-relax)
3. **Magnetic systems** (nspin = 2)
4. **DFT+U** for correlated systems
5. **GW calculations** for accurate band gaps
6. **Molecular dynamics** (cp.x)

---

## Congratulations!

You have completed the Quantum ESPRESSO Workshop. You now have the foundation to:

✓ Set up and run DFT calculations independently  
✓ Perform rigorous convergence testing  
✓ Calculate and interpret electronic structure  
✓ Troubleshoot common issues  
✓ Extend to new materials and properties  

**Happy computing!**